<a href="https://colab.research.google.com/github/njucs/RelationExtraction/blob/main/CRE/ACL2019_tf.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Python 2.7 + Tensorflow 1.14.0
!python --version
!whereis python2
#!python -m pip uninstall pip
#!update-alternatives --remove python /usr/local/bin/python
!update-alternatives --install /usr/local/bin/python python /usr/bin/python2
#!wget https://bootstrap.pypa.io/get-pip.py
#!python get-pip.py

!python --version
#!pip install tensorflow-gpu==1.14

Python 3.7.13
python2: /usr/bin/python2.7-config /usr/bin/python2 /usr/bin/python2.7 /usr/lib/python2.7 /etc/python2.7 /usr/local/lib/python2.7 /usr/include/python2.7 /usr/share/man/man1/python2.1.gz
update-alternatives: --install needs <link> <name> <path> <priority>

Use 'update-alternatives --help' for program usage information.
Python 3.7.13


#### Load Data

In [ ]:
import sys
import os
import pandas as pd
import numpy as np
# import cPickle
import _pickle as cPickle
from numpy import array
import operator
import random
from fuzzywuzzy import fuzz

import tensorflow as tf
#tf.enable_eager_execution()
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

import unicodedata
import re
import time

print(tf.__version__)

DEVICE = "0"
os.environ["CUDA_VISIBLE_DEVICES"]=DEVICE

INPUT_LENGTH = 32
OUTPUT_LENGTH = 14
TOTAL_EPOCHS = 20
BEAMSEARCH = True
POSPROCESSING = True

2.4.1


In [ ]:
word_to_idx = cPickle.load(open("data/ready/encoder-decoder-model-training/source_embedding_dict.pickle", "rb"))
#idx_to_word = {v: k for k, v in word_to_idx.iteritems()}
idx_to_word = {v: k for k, v in word_to_idx.items()}

ent_to_idx = cPickle.load(open("data/ready/encoder-decoder-model-training/target_embedding_dict.pickle", "rb"))
#idx_to_ent = {v: k for k, v in ent_to_idx.iteritems()}
idx_to_ent = {v: k for k, v in ent_to_idx.items()}

with open("data/ready/encoder-decoder-model-training/source_embedding_matrix.pickle", "rb") as f:
    source_embedding_matrix = cPickle.load(f, encoding="bytes")

with open("data/ready/encoder-decoder-model-training/target_embedding_matrix.pickle", "rb") as f:
    target_embedding_matrix = cPickle.load(f, encoding="bytes")
    
#source_embedding_matrix = cPickle.load(open("data/ready/encoder-decoder-model-training/source_embedding_matrix.pickle", "rb"))
#target_embedding_matrix = cPickle.load(open("data/ready/encoder-decoder-model-training/target_embedding_matrix.pickle", "rb"))

train_dataset = cPickle.load(open("data/ready/encoder-decoder-model-training/train_dataset.pickle", "rb"))
test_dataset = cPickle.load(open("data/ready/encoder-decoder-model-training/test_dataset.pickle", "rb"))
val_dataset = cPickle.load(open("data/ready/encoder-decoder-model-training/val_dataset.pickle", "rb"))
trip_dataset = cPickle.load(open("data/ready/encoder-decoder-model-training/trip_dataset.pickle", "rb"))

pred_dict = cPickle.load(open("data/ready/pred_to_idx.pickle", "rb"))

In [ ]:
with open("data/ready/dictionary.pickle", "rb") as f:
    dictionary = cPickle.load(f, encoding="bytes")

with open("data/ready/wikidata_link.pickle", "rb") as f:
    wikidata_to_dbp = cPickle.load(f, encoding="bytes")

#dictionary = cPickle.load(open("data/ready/dictionary.pickle","rb"))
#wikidata_to_dbp = cPickle.load(open("data/ready/wikidata_link.pickle","rb"))

In [ ]:
training_input = list()
training_output = list()
for t in train_dataset:
    input_data, output_data, ner_data = t
    
    input_data = input_data.split()
    input_data = [word_to_idx[x] for x in input_data]
    input_data.insert(0, word_to_idx["<START>"])
    input_data.append(word_to_idx["<END>"])
    sent_array = [0] * INPUT_LENGTH
    sent_array[:len(input_data)] = input_data
    training_input.append(sent_array)
    
    output_data = [ent_to_idx[x] for x in output_data]
    output_data.insert(0, ent_to_idx["<START>"])
    output_data.append(ent_to_idx["<END>"])
    target_array = [0] * OUTPUT_LENGTH
    target_array[:len(output_data)] = output_data
    training_output.append(target_array)
    
testing_input = list()
testing_output = list()
for t in test_dataset:
    input_data, output_data, ner_data = t
    
    input_data = input_data.split()
    input_data = [word_to_idx[x] for x in input_data]
    input_data.insert(0, word_to_idx["<START>"])
    input_data.append(word_to_idx["<END>"])
    sent_array = [0] * INPUT_LENGTH
    sent_array[:len(input_data)] = input_data
    testing_input.append(sent_array)
    
    output_data = [ent_to_idx[x] for x in output_data]
    output_data.insert(0, ent_to_idx["<START>"])
    output_data.append(ent_to_idx["<END>"])
    target_array = [0] * OUTPUT_LENGTH
    target_array[:len(output_data)] = output_data
    testing_output.append(target_array)
    
    
validation_input = list()
validation_output = list()
for t in val_dataset:
    input_data, output_data, ner_data = t
    
    input_data = input_data.split()
    input_data = [word_to_idx[x] for x in input_data]
    input_data.insert(0, word_to_idx["<START>"])
    input_data.append(word_to_idx["<END>"])
    sent_array = [0] * INPUT_LENGTH
    sent_array[:len(input_data)] = input_data
    validation_input.append(sent_array)
    
    output_data = [ent_to_idx[x] for x in output_data]
    output_data.insert(0, ent_to_idx["<START>"])
    output_data.append(ent_to_idx["<END>"])
    target_array = [0] * OUTPUT_LENGTH
    target_array[:len(output_data)] = output_data
    validation_output.append(target_array)
    
trip_input = list()
trip_output = list()
for t in trip_dataset:
    input_data, output_data, ner_data = t
    
    input_data = input_data.split()
    input_data = [word_to_idx[x] for x in input_data]
    input_data.insert(0, word_to_idx["<START>"])
    input_data.append(word_to_idx["<END>"])
    sent_array = [0] * INPUT_LENGTH
    sent_array[:len(input_data)] = input_data
    trip_input.append(sent_array)
    
    output_data = [ent_to_idx[x] for x in output_data]
    output_data.insert(0, ent_to_idx["<START>"])
    output_data.append(ent_to_idx["<END>"])
    target_array = [0] * OUTPUT_LENGTH
    target_array[:len(output_data)] = output_data
    trip_output.append(target_array)
    
# Name Expansion
name_training_input = list()
name_training_output = list()
for e in ent_to_idx:
    if e in wikidata_to_dbp:
        ent_name = wikidata_to_dbp[e].split("/")[-1].replace("_", " ").split()
        in_embedding = True
        for token in ent_name:
            if token not in word_to_idx:
                in_embedding = False
        if not in_embedding:
            in_embedding = True
            ent_name = dictionary[e].split()
            for token in ent_name:
                if token not in word_to_idx:
                    in_embedding = False
        if in_embedding:
            source = list()
            source.append(word_to_idx["<START>"])
            for token in ent_name:
                source.append(word_to_idx[token])
            source.append(word_to_idx["<END>"])
            target = [ent_to_idx["<START>"], ent_to_idx[e], ent_to_idx["<END>"]]
            source[len(source):INPUT_LENGTH] = [0] * (INPUT_LENGTH-len(source))
            target[len(target):OUTPUT_LENGTH] = [0] * (OUTPUT_LENGTH-len(target))
            name_training_input.append(source)
            name_training_output.append(target)
    else:
        try:
            ent_name = dictionary[e].split()
            in_embedding = True
            for token in ent_name:
                if token not in word_to_idx:
                    in_embedding = False
            if in_embedding:
                source = list()
                source.append(word_to_idx["<START>"])
                for token in ent_name:
                    source.append(word_to_idx[token])
                source.append(word_to_idx["<END>"])
                target = [ent_to_idx["<START>"], ent_to_idx[e], ent_to_idx["<END>"]]
                source[len(source):INPUT_LENGTH] = [0] * (INPUT_LENGTH-len(source))
                target[len(target):OUTPUT_LENGTH] = [0] * (OUTPUT_LENGTH-len(target))
                name_training_input.append(source)
                name_training_output.append(target)
        except:
            pass
name_training_input += training_input
name_training_output += training_output
training_input = name_training_input
training_output = name_training_output


training_input = array(training_input, dtype=np.int32)
training_output = array(training_output, dtype=np.int32)
testing_input = array(testing_input, dtype=np.int32)
testing_output = array(testing_output, dtype=np.int32)
validation_input = array(validation_input, dtype=np.int32)
validation_output = array(validation_output, dtype=np.int32)
trip_input = array(trip_input, dtype=np.int32)
trip_output = array(trip_output, dtype=np.int32)


print('training size', len(training_input))
print('validation size', len(validation_input))
print('test size', len(testing_input))
print('trip size', len(trip_input))

training size 224881
validation size 988
test size 29785
trip size 1000


#### Build MODEL

In [ ]:
BUFFER_SIZE = len(training_input)
BATCH_SIZE = 64
N_BATCH = BUFFER_SIZE//BATCH_SIZE
embedding_dim = 64
units = 512
vocab_inp_size = len(word_to_idx)
vocab_tar_size = len(ent_to_idx)

train_dataset = tf.data.Dataset.from_tensor_slices((training_input, training_output)).shuffle(BUFFER_SIZE)
train_dataset = train_dataset.apply(tf.contrib.data.batch_and_drop_remainder(BATCH_SIZE))
val_dataset = tf.data.Dataset.from_tensor_slices((validation_input, validation_output)).shuffle(BUFFER_SIZE)
val_dataset = val_dataset.apply(tf.contrib.data.batch_and_drop_remainder(BATCH_SIZE))

AttributeError: module 'tensorflow' has no attribute 'contrib'

In [ ]:
def gru(units):
    if tf.test.is_gpu_available():
        return tf.keras.layers.CuDNNGRU(units, return_sequences=True, return_state=True, recurrent_initializer='glorot_uniform')
    else:
        return tf.keras.layers.GRU(units, return_sequences=True, return_state=True, recurrent_activation='sigmoid', recurrent_initializer='glorot_uniform')

In [ ]:
#### ENCODER
class Encoder(tf.keras.Model):
    def __init__(self, vocab_size, embedding_dim, enc_units, batch_sz):
        super(Encoder, self).__init__()
        self.batch_sz = batch_sz
        self.enc_units = enc_units
        self.embeddings_initializer = tf.constant_initializer(source_embedding_matrix)
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim, mask_zero=True, trainable=True)
        
        self.gru = gru(self.enc_units)
        
    def get_ngram_tensor(self, stacked_tensor, N):
        unstacked_tensor = tf.unstack(stacked_tensor, stacked_tensor.get_shape().as_list()[1], 1)
        result = list()
        for i in range(len(unstacked_tensor)-(N-1)):
            tmp_tensor = unstacked_tensor[i]
            for j in range(1, N):
                tmp_tensor = tf.concat([tmp_tensor, unstacked_tensor[j+i]], 1)
            result.append(tmp_tensor)
        result = tf.stack(result, 1)
        return result
    
    def call(self, x, hidden):
        x = self.embedding(x)
        output, state = self.gru(x, initial_state = hidden)
        
        embedded_input = x
        return output, state, embedded_input
    
    def initialize_hidden_state(self):
        return tf.zeros((self.batch_sz, self.enc_units))

In [ ]:
#### DECODER
class Decoder(tf.keras.Model):
    def __init__(self, vocab_size, embedding_dim, dec_units, batch_sz):
        super(Decoder, self).__init__()
        self.batch_sz = batch_sz
        self.dec_units = dec_units
        self.embeddings_initializer = tf.constant_initializer(target_embedding_matrix)
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim, mask_zero=True, trainable=True)
        self.gru = gru(self.dec_units)
        self.fc = tf.keras.layers.Dense(vocab_size)
        
        # used for attention
        self.W1 = tf.keras.layers.Dense(self.dec_units)
        self.W2 = tf.keras.layers.Dense(self.dec_units)
        self.V = tf.keras.layers.Dense(1)
        
        self.W1_1 = tf.keras.layers.Dense(self.dec_units)
        self.V_1 = tf.keras.layers.Dense(1)
        
        self.W1_2 = tf.keras.layers.Dense(self.dec_units)
        self.V_2 = tf.keras.layers.Dense(1)
        
        self.W1_3 = tf.keras.layers.Dense(self.dec_units)
        self.V_3 = tf.keras.layers.Dense(1)
        
        self.W_cv1 = tf.keras.layers.Dense(self.dec_units)
        self.W_cv2 = tf.keras.layers.Dense(self.dec_units)
        self.W_cv3 = tf.keras.layers.Dense(self.dec_units)
        
    def get_ngram_tensor(self, stacked_tensor, N):
        unstacked_tensor = tf.unstack(stacked_tensor, stacked_tensor.get_shape().as_list()[1], 1)
        result = list()
        for i in range(len(unstacked_tensor)-(N-1)):
            tmp_tensor = unstacked_tensor[i]
            for j in range(1, N):
                tmp_tensor = tf.concat([tmp_tensor, unstacked_tensor[j+i]], 1)
            result.append(tmp_tensor)
        result = tf.stack(result, 1)
        return result
        
    def call(self, x, hidden, enc_output, embedded_input):
        
        hidden_with_time_axis = tf.expand_dims(hidden, 1)
        score = self.V(tf.nn.tanh(self.W1(enc_output) + self.W2(hidden_with_time_axis)))
        attention_weights = tf.nn.softmax(score, axis=1)
        context_vector = attention_weights * enc_output
        context_vector = tf.reduce_sum(context_vector, axis=1)
        
        x = self.embedding(x)
        
        enc_output_1 = self.get_ngram_tensor(embedded_input, 1)
        score_1 = self.V_1(tf.nn.tanh(self.W1_1(enc_output_1) + self.W2(hidden_with_time_axis)))
        attention_weights_1 = tf.nn.softmax(score_1, axis=1)
        context_vector_1 = attention_weights_1 * enc_output_1
        context_vector_1 = tf.reduce_sum(context_vector_1, axis=1)
        context_vector_1 = self.W_cv1(context_vector_1)

        enc_output_2 = self.get_ngram_tensor(embedded_input, 2)
        score_2 = self.V_2(tf.nn.tanh(self.W1_2(enc_output_2) + self.W2(hidden_with_time_axis)))
        attention_weights_2 = tf.nn.softmax(score_2, axis=1)
        context_vector_2 = attention_weights_2 * enc_output_2
        context_vector_2 = tf.reduce_sum(context_vector_2, axis=1)
        context_vector_2 = self.W_cv2(context_vector_2)

        enc_output_3 = self.get_ngram_tensor(embedded_input, 3)
        score_3 = self.V_3(tf.nn.tanh(self.W1_3(enc_output_3) + self.W2(hidden_with_time_axis)))
        attention_weights_3 = tf.nn.softmax(score_3, axis=1)
        context_vector_3 = attention_weights_3 * enc_output_3
        context_vector_3 = tf.reduce_sum(context_vector_3, axis=1)
        context_vector_3 = self.W_cv3(context_vector_3)

        context_vector_n = tf.nn.tanh(context_vector_1 + context_vector_2 + context_vector_3)

        x = tf.concat([tf.expand_dims(context_vector_n, 1), tf.expand_dims(context_vector, 1), x], axis=-1)
        
        # passing the concatenated vector to the GRU
        output, state = self.gru(x)
        output = tf.reshape(output, (-1, output.shape[2]))
        x = self.fc(output)
        
        return x, state, attention_weights
        
    def initialize_hidden_state(self):
        return tf.zeros((self.batch_sz, self.dec_units))

In [ ]:
encoder = Encoder(vocab_inp_size, embedding_dim, units, BATCH_SIZE)
decoder = Decoder(vocab_tar_size, embedding_dim, units, BATCH_SIZE)

In [ ]:
optimizer = tf.train.AdamOptimizer(0.0002)
def loss_function(real, pred):
    mask = 1 - np.equal(real, 0)
    loss_ = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=real, logits=pred) * mask
    return tf.reduce_mean(loss_)

In [ ]:
checkpoint_dir = 'data/ready/encoder-decoder-model-training/model/'
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt")
checkpoint = tf.train.Checkpoint(optimizer=optimizer, encoder=encoder, decoder=decoder)

In [ ]:
EPOCHS = TOTAL_EPOCHS

log_file = open("log", "w", 0)

for epoch in range(EPOCHS):
    start = time.time()
    total_loss = 0
    for (batch, (inp, targ)) in enumerate(train_dataset):
        hidden = encoder.initialize_hidden_state()
        loss = 0

        with tf.GradientTape() as tape:
            enc_output, enc_hidden, embedded_input = encoder(inp, hidden)
            dec_hidden = enc_hidden
            dec_input = tf.expand_dims([word_to_idx['<START>']] * BATCH_SIZE, 1)       

            for t in range(1, targ.shape[1]):
                predictions, dec_hidden, _ = decoder(dec_input, dec_hidden, enc_output, embedded_input)
                loss += loss_function(targ[:, t], predictions)
                dec_input = tf.expand_dims(targ[:, t], 1)

        batch_loss = (loss / int(targ.shape[1]))
        total_loss += batch_loss
        variables = encoder.variables + decoder.variables
        gradients = tape.gradient(loss, variables)
        optimizer.apply_gradients(zip(gradients, variables))

        if batch % 100 == 0:
            log_str = ('Epoch {} Batch {} Loss {:.4f}'.format(epoch + 1, batch, batch_loss.numpy()))
            print log_str
            log_file.write(log_str+"\n")
    
    if (epoch + 1) % 10 == 0:
        checkpoint.save(file_prefix = checkpoint_prefix)

    val_loss = 0
    for (batch, (inp, targ)) in enumerate(val_dataset):
        hidden = encoder.initialize_hidden_state()
        loss = 0
        with tf.GradientTape() as tape:
            enc_output, enc_hidden, embedded_input = encoder(inp, hidden)
            dec_hidden = enc_hidden
            dec_input = tf.expand_dims([word_to_idx['<START>']] * BATCH_SIZE, 1)       

            for t in range(1, targ.shape[1]):
                predictions, dec_hidden, _ = decoder(dec_input, dec_hidden, enc_output, embedded_input)
                loss += loss_function(targ[:, t], predictions)
                dec_input = tf.expand_dims(targ[:, t], 1)
        batch_loss = (loss / int(targ.shape[1]))
        val_loss += batch_loss

    log_str = ('Epoch {} Loss {:.4f}'.format(epoch + 1, total_loss / N_BATCH))
    print log_str
    log_file.write(log_str+"\n")
    log_str = ('Epoch {} Val Loss {:.4f}'.format(epoch + 1, val_loss / (len(validation_input)//BATCH_SIZE)))
    print log_str
    log_file.write(log_str+"\n")
    log_str = ('Time taken for 1 epoch {} sec\n'.format(time.time() - start))
    print log_str
    log_file.write(log_str+"\n")
print "DONE"
log_file.write("DONE\n")
log_file.close()

#### Testing

In [ ]:
# LOAD EXISTING MODEL
checkpoint.restore(tf.train.latest_checkpoint(checkpoint_dir))

In [ ]:
def get_gold_standard(expected_output):
    result = []
    for e in expected_output:
        result.append(idx_to_ent[e])
    return result

In [ ]:
clf = cPickle.load(open("data/ready/triple_classifier_model/model.pickle", "rb"))

def one_hot_encoding(pred, pred_dict):
    enc = [0]*len(pred_dict)
    enc[pred_dict[pred]] = 1
    return enc

def get_sent(result, prob_result, postprocessing):
    sent_set = set()
    sent = list()
    sent_id = list()
    for i,token in enumerate(result):
        if token == "<START>":
            continue
        if token == "<END>":
            if len(sent) == 3:
                sent_set.add(" ### ".join(sent))
            return sent_set
        sent.append(dictionary[token])
        sent_id.append(token)
        if len(sent) == 3:
            if postprocessing:
                if (prob_result[i]+prob_result[i-1]+prob_result[i-2])/3 < 0.70:
                    s = sent_id[0]
                    p = sent_id[1]
                    o = sent_id[2]

                    if s.startswith("Q") and p.startswith("P") and o.startswith("Q"):

                        embedding_s = np.asarray(target_embedding_matrix[ent_to_idx[s]]).astype(np.float)
                        embedding_o = np.asarray(target_embedding_matrix[ent_to_idx[o]]).astype(np.float)
                        embedding_p = np.asarray(target_embedding_matrix[ent_to_idx[p]]).astype(np.float)
                        diff = (abs(embedding_s+embedding_p-embedding_o)).sum()
                        valid_triple = clf.predict([one_hot_encoding(p, pred_dict)+ one_hot_encoding(TEST_DATASET, {"seen":0,"nyt":1,"trip":2})+[diff]])[0]
                        if valid_triple == 0:
                            sent_set.add(" ### ".join(sent))
                else:
                    sent_set.add(" ### ".join(sent))
            else:
                sent_set.add(" ### ".join(sent))
            sent = list()
    return sent_set

In [ ]:
def get_sent_from_training_data(training_data):
    sent = ""
    for tok in training_data:
        sent += idx_to_word[tok]+" "
        if idx_to_word[tok] == "<END>":
            return sent.strip() 
    return sent.strip()

In [ ]:
from kitchen.text.converters import getwriter, to_bytes, to_unicode
from kitchen.i18n import get_translation_object
translations = get_translation_object('example')
_ = translations.ugettext
b_ = translations.lgettext

def phrase_sim(sent, phrases):
    result = list()
    pos = 0
    for p in phrases:
        l = len(p.split())
        splitted = sent.split()
        max_score = 0
        for i in range(len(splitted)-l+1):
            test = " ".join(splitted[i:i+l])
            score = fuzz.ratio(p, test)
            if score > max_score:
                max_score = score
        result.append((p, len(phrases)-pos, max_score))
        pos += 1
    result = sorted(result, key = lambda x: (x[2], x[1],len(x[0])), reverse=True)
    return result

def generate_output(sentence, encoder, decoder):
    attention_plot = np.zeros((OUTPUT_LENGTH, INPUT_LENGTH))
    sentence_ori = get_sent_from_training_data(sentence[0])
    result = list()
    prob_result = list()
    inputs = tf.convert_to_tensor(sentence)
    hidden = [tf.zeros((1, units))]
    enc_out, enc_hidden, embedded_input = encoder(inputs, hidden)

    dec_hidden = enc_hidden
    dec_input = tf.expand_dims([ent_to_idx['<START>']], 0)

    if BEAMSEARCH:
        for t in range(OUTPUT_LENGTH):
            if t%3==0 or t%3==2:
                beam = True
            else:
                beam = False
            predictions, dec_hidden, attention_weights = decoder(dec_input, dec_hidden, enc_out, embedded_input)
            prob_predictions = tf.nn.softmax(predictions)
            attention_weights = tf.reshape(attention_weights, (-1, ))
            attention_plot[t] = attention_weights.numpy()

            predicted_id = tf.argmax(predictions[0]).numpy()
            if prob_predictions[0][predicted_id] > 0.99:
                beam = False
            
            if idx_to_ent[predicted_id] == '<END>':
                return result,prob_result,sentence_ori, attention_plot

            if beam:
                if t%3==2:
                    emb_s = np.asarray(target_embedding_matrix[ent_to_idx[result[t-2]]])
                    emb_p = np.asarray(target_embedding_matrix[ent_to_idx[result[t-1]]])
                    diff = tf.cast(tf.nn.softmax(-np.sum(abs(emb_s+emb_p-target_embedding_matrix), axis=1), axis=0), tf.float32)
                    prob_predictions_tmp = tf.multiply(prob_predictions[0], diff)
                    cand_predicted_id = tf.nn.top_k(prob_predictions_tmp, k=100, sorted=True)[1].numpy()
                else:
                    cand_predicted_id = tf.nn.top_k(predictions[0], k=10, sorted=True)[1].numpy()
                
                phrases = list()
                rev_dict = dict()
                for p in cand_predicted_id:
                    if idx_to_ent[p] == "<END>" or idx_to_ent[p] == "<START>":
                        continue
                    phrases.append(_(dictionary[idx_to_ent[p]]))
                    rev_dict[dictionary[idx_to_ent[p]]] = idx_to_ent[p]
                predicted_id = np.int64(ent_to_idx[rev_dict[b_(phrase_sim(sentence_ori, phrases)[0][0])]])
            else:
                predicted_id = tf.argmax(predictions[0]).numpy()
                
            result.append(idx_to_ent[predicted_id])
            prob_result.append(prob_predictions[0][predicted_id])
            dec_input = tf.expand_dims([predicted_id], 0)
    else:
        for t in range(OUTPUT_LENGTH):
            predictions, dec_hidden, attention_weights = decoder(dec_input, dec_hidden, enc_out, embedded_input)
            prob_predictions = tf.nn.softmax(predictions)
            attention_weights = tf.reshape(attention_weights, (-1, ))
            attention_plot[t] = attention_weights.numpy()
            
            predicted_id = tf.argmax(predictions[0]).numpy()

            if idx_to_ent[predicted_id] == '<END>':
                return result,prob_result,sentence_ori, attention_plot
            
            result.append(idx_to_ent[predicted_id])
            prob_result.append(prob_predictions[0][predicted_id])
            dec_input = tf.expand_dims([predicted_id], 0)

    return result,prob_result,sentence_ori, attention_plot

In [ ]:
INSPECT = True
TEST_DATASET = "seen" # 'trip' or 'seen'

log_file = open("result_log", "w", 0)

if TEST_DATASET == "nyt":
    test_in = nyt_input
    test_out = nyt_output
elif TEST_DATASET == "seen":
    test_in = testing_input
    test_out = testing_output
elif TEST_DATASET == "trip":
    test_in = trip_input
    test_out = trip_output

if INSPECT:
    len_test = 100
else:
    len_test = len(test_in)

num_correct = 0
total_prediction = 0
total_gold_standard = 0
eval_dataset = (test_in[:len_test], test_out[:len_test])
for i, test_sent in enumerate(eval_dataset[0]):
    if (i+1) % 100 == 0:
        print i,str(num_correct/float(total_prediction)),str(num_correct/float(total_gold_standard))
        log_file.write(str(i)+" "+str(num_correct/float(total_prediction))+" "+str(num_correct/float(total_gold_standard))+"\n")
    result,prob_result,sentence, attention_plot = generate_output([test_sent], encoder, decoder)
    result_sent = get_sent(result[:], prob_result, POSPROCESSING)
    gold_standard = get_gold_standard(eval_dataset[1][i])
    gold_standard_sent = get_sent(gold_standard,None, False)


    if INSPECT:
        if result_sent != gold_standard_sent:
            print i
            print "Input: ", sentence
            print "Predicted: ", result, result_sent
            print "Expected: ", gold_standard, gold_standard_sent
            print
        if i > 100:
            break

    total_prediction += len(result_sent)
    total_gold_standard += len(gold_standard_sent)

    for s in result_sent:
        if s in gold_standard_sent:
            num_correct += 1
print "PRECISION: ", num_correct/float(total_prediction)
print "RECALL: ", num_correct/float(total_gold_standard)
log_file.write("PRECISION: "+str(num_correct/float(total_prediction))+"\n")
log_file.write("RECALL: "+str(num_correct/float(total_gold_standard))+"\n")